<a href="https://colab.research.google.com/github/andersknudby/Teledetection/blob/master/GEE%20Labo%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GEE Labo 3 : Séries chronologiques et graphiques avec MODIS

## Introduction

Dans ce laboratoire, nous allons continuer à travailler avec l'API python de Google Earth Engine dans Google Colab. Nous explorerons les jeux de données MODIS disponibles et créerons une série chronologique de température de surface de la mer (SST) avec MODIS. Nous explorerons également certaines bibliothèques graphiques python pour créer des graphiques interactifs des données du moteur Earth.


**Fichiers dont vous avez besoin:** N/A

**Préparation:** Rien

## Étapes

### 1. Importations, installation, authentification, initialisation

Notre première étape est de mettre en place le notebook. Revoyez la description dans le premier laboratoire si vous voulez un rappel sur pourquoi nous devons faire cela !

In [ ]:
# Install geemap library so that we can use it to view images on an interactive map
## You may get a runtime warning - you can ignore that
!pip install geemap

In [ ]:
# Import the necessary libraries
import ee
import numpy as np
import geemap.eefolium as geemap
import pprint

import pandas as pd
import altair as alt
import folium

# Set up a 'pretty printer' to print ...
pp = pprint.PrettyPrinter(depth=3)

In [ ]:
# Authenticate and initialize this instance of GEE in Google Colab
## Follow the prompts and fill in authentication code
ee.Authenticate()
ee.Initialize()

### 2. MODIS

Jusqu'à présent, dans ces laboratoires d'GEE, nous avons presque exclusivement utilisé des données Landsat. Les données Landsat conviennent bien à l'apprentissage de GEE car elles possèdent des produits TOA et de réflectance de surface bien connus qui couvrent une très longue période, ont une bonne résolution spectrale et une résolution spatiale décente pour de nombreuses applications. Pour cette raison, Google et d'autres utilisateurs de GEE disposent d'une documentation abondante sur l'utilisation de l'imagerie Landsat dans GEE. 

Cependant, l'un des principaux avantages de l'utilisation de GEE est la facilité d'accès à d'énormes quantités de *différents types de données*. GEE combine des collections entières d'images et de produits d'imagerie provenant de divers satellites et organisations et permet de les analyser facilement, sans avoir à télécharger des images individuelles à partir de la source.

Ainsi, pour mieux comprendre la puissance de Google Earth Engine, et pour vous exposer à d'autres codes que vous pourrez utiliser à l'avenir, pour ce laboratoire, nous allons examiner des données à résolution spatiale modérée et à haute résolution temporelle avec MODIS.

#### MODIS Info

MODIS (ou Moderate Resolution Imaging Spectroradiometer) est un instrument embarqué à bord de deux satellites complémentaires lancés par la NASA : **Terra** et **Aqua**. Combinés, les instruments MODIS Terra et Aqua prennent des images de toute la surface de la Terre tous les 1 à 2 jours, acquérant des données dans 36 bandes spectrales à une résolution de 250 m (bandes 1-2), 500 m (bandes 3-7) et 1000 m (bandes 8-36). 
 
Grâce à leur faible résolution spatiale mais à leur haute résolution temporelle, les données MODIS sont utiles pour suivre l'évolution du paysage dans le temps.

Comme Landsat, MODIS dispose de produits de données en plus de l'imagerie à 36 bandes à laquelle vous pouvez accéder à partir du catalogue GEE. Consultez toutes les données disponibles dans la [section MODIS du catalogue GEE](https://developers.google.com/earth-engine/datasets/catalog/modis).

La structure des données MODIS est un peu plus complexe que ce que nous avons traité jusqu'à présent. Les données de MODIS sont traitées par divers groupes (selon le produit) afin de créer les produits de données disponibles. Contrairement à d'autres données satellitaires, MODIS ne fournit pas de scènes non traitées au grand public.

#### Utilisation de la réflectance de surface MODIS
Pour commencer, nous allons utiliser le produit de données quotidiennes MOD09GA.006 Terra Surface Reflectance Daily Global 1km and 500m. Cette collection a une réflectance de surface disponible à une résolution de 500m, ainsi que des informations sur les bits d'assurance qualité (QA) des pixels à la résolution de 1000m. Ces bits d'assurance qualité contiennent des informations sur la contamination des nuages, ainsi que le nombre d'observations qui composent un pixel donné - n'oubliez pas que ces pixels sont des produits de données*** et qu'ils peuvent donc représenter plus d'une observation.

Le masquage des pixels non désirés se fait de la même manière que pour Landsat, en utilisant les bandes QA des pixels MODIS. Dans ce cas, nous allons créer des fonctions pour masquer à la fois les pixels de nuages (à partir de la bande QA `state_1km`) et les pixels sans observations (à partir de la bande `num_observation_1km`). 

In [ ]:
#Calculate how frequently a location is labeled as clear (i.e. non-cloudy)
#according to the "internal cloud algorithm flag" of the MODIS "state 1km"
#QA band.

# A function to mask out pixels that did not have observations.
def maskEmptyPixels(image):
  # Find pixels with >0 observations and mask out all other pixels 
  withObs = image.select('num_observations_1km').gt(0)
  return image.updateMask(withObs)

# A function to mask out cloudy pixels.
def maskClouds(image):
  #Select the QA band.
  QA = image.select('state_1km')
  # Make a mask to get bit 10, the internal_cloud_algorithm_flag bit.
  bitMask = 1 << 10
  # Return an image masking out cloudy areas.
  return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))


Maintenant, appliquons les fonctions de masquage à une collection et affichons-la sur la carte :

In [ ]:
# Start with an image collection for summer 2010 and apply masks
summer2010 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2010-04-01', '2010-09-30')\
        .map(maskEmptyPixels)\
        .map(maskClouds)

# note that slashes \ are used in the code above to allow another line of code for 
# formatting appearance (the code could all be on one long line, if you wanted)
# in some cases, code automatically continues to the next line, e.g. when there are commas,
# but sometimes we need to tell the code that we are continuing on the next line with a \

# Get the median pixel value for summer 2010
# Recall from the last lab that this is a temporal reducer
summer2010med = summer2010.median()

# Note that true colour for this MODIS dataset is Band 1 in Red, Band 4 in Green, and Band 3 in Blue
visParams = {'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
     'gain': 0.07,
     'gamma': 1.4
    }

Map = geemap.Map()
Map.addLayer(summer2010med,visParams,'Summer 2010 Median SR')
Map.addLayerControl()
Map.setCenter(41.92,32.36,5)
Map

Remarquez que nous n'avons pas filtré les limites de l'image et que nous avons donc créé une médiane pour l'ensemble du globe (où il existe des données pour l'été 2010). Vous pouvez donc faire un panoramique et zoomer sur la zone de votre choix pour voir à quoi ressemblent les données.

En raison de la fréquence temporelle élevée de MODIS, l'imagerie résultante est assez grande, car le pixel médian est potentiellement calculé à partir de 182 pixels. C'est-à-dire un pixel par jour pour chaque jour du 1er avril au 30 septembre (182 jours).

#### Utilisation d'autres produits de données MODIS

En plus des produits de réflectance de surface, MODIS fournit également des produits spéciaux calculés à partir des données. Pour le prochain exemple, nous utiliserons les données MODIS Aqua de l'image cartographique standard de la couleur de l'océan. 

Ce jeu de données est un produit de niveau 3 qui comprend des données sur la couleur des océans et la biologie des océans par satellite produites par le programme Earth Science Data Systems (ESDS) de la NASA. Il comprend des bandes qui sont utiles pour calculer les données de biologie océanique en raison de leurs longueurs d'onde étroites, ainsi que certaines bandes de données pré-calculées telles que la concentration en chlorophylle-a et la température de surface de la mer (SST).

Examinons la bande SST et utilisons-la pour étudier les années El Nino et La Nina. Si vous avez besoin de vous rafraîchir la mémoire, consultez les pages de la NASA consacrées aux sciences de l'enfant pour [El Nino](https://spaceplace.nasa.gov/el-nino/en/) et [La Nina](https://spaceplace.nasa.gov/la-nina/en/).

In [ ]:
# Load MODIS Ocean Colour Image Collection
modisOceanColor = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI');

# Create a collection from the SST (sea surface temperature) band
sst = modisOceanColor.select(['sst'])

# Calculate median SST values for winter 2015/2016 - an El Nino year
sstElNino = sst.filterDate('2015-12-01', '2016-02-28').median()

# Calculate median SST values for winter 2010/2011 - a La Nina year
sstLaNina = sst.filterDate('2010-12-01', '2011-02-28').median()


In [ ]:
# Create vis parameters for SST with a palette that goes from blue to red
vis = {'min': -4, 'max': 35, 'palette': '#2166AC,#4393C3,#92C5DE,#D1E5F0,#B2182B'};

Map = geemap.Map()
Map.addLayer(sstElNino,vis,'El Nino')
Map.addLayer(sstLaNina,vis,'La Nina')
# set the map centre as the middle of the Pacific at a zoom level of two (pretty zoomed out)
Map.setCenter(-146.25,0,2)
Map.addLayerControl()
Map

Activez et désactivez les couches pour comparer la température de surface de la mer pour les années El Nino et La Nina. 

Rappelez-vous de vos cours de climatologie (ou des pages NASA Kids liées ci-dessus) qu'El Nino résulte de vents commerciaux plus faibles se déplaçant d'est en ouest, ce qui entraîne des eaux plus chaudes vers la côte ouest de l'Amérique centrale et du Sud. La Nina se produit les années où les alizés sont plus forts que la normale, ce qui entraîne des eaux plus froides au large de la côte ouest de l'Amérique centrale et du Sud.

L'effet d'El Niño-Southern Oscillation (ENSO) est mieux visualisé sous la forme d'une anomalie de température afin de montrer l'effet sur les zones qui ont encore des eaux relativement fraîches, plutôt que sous la forme d'une température absolue qui montre une eau de 10 degrés en bleu clair, indépendamment du fait que 10 degrés soient plus froids ou plus chauds que les conditions normales. Cependant, ces deux couches montrent bien comment MODIS peut être utilisé pour examiner des modèles de données à grande échelle. 

### 3. Graphiques de séries temporelles avec MODIS


L'analyse des séries chronologiques consiste à examiner les données dans le temps pour détecter et décrire les changements. Avec l'imagerie, cela peut se faire sous la forme d'une vidéo ou, plus simplement, en extrayant certaines données de l'imagerie et en affichant l'évolution dans le temps sur un graphique.

L'éditeur de code du moteur Google Earth dispose d'une bibliothèque graphique intégrée, appelée Chart.UI, qui intègre certaines fonctionnalités du moteur Earth, comme les réducteurs, dans des fonctions graphiques permettant de créer facilement des visualisations de données.

Malheureusement, Chart.UI ne fonctionne pas dans l'API python, nous devons donc créer des fonctions de réduction que nous pouvons appliquer aux collections d'images avant de les tracer avec des bibliothèques graphiques python normales. Nous allons examiner deux bibliothèques de graphiques dans ce laboratoire : la simple **matplotlib**, et **Altair**, qui est une bibliothèque utile pour créer de belles cartes interactives.

#### Créer une fonction réductrice

Pour créer une analyse de série temporelle, nous pouvons soit examiner l'évolution des valeurs des pixels dans le temps pour un **pixel unique** (à un point défini), soit examiner l'évolution des valeurs dans le temps pour une **région de pixels** (dans une géométrie définie). Dans les deux cas, nous devons utiliser un réducteur pour résumer les valeurs d'une zone avant de tracer la série chronologique. 

Comme mentionné, la bibliothèque Chart.UI intègre cette capacité de réduction dans ses fonctions graphiques, mais dans l'API Python de colab, nous devons définir les fonctions qui appliqueront un réducteur à une collection d'images avant de tracer la série chronologique. Les fonctions ci-dessous sont tirées directement d'un [GEE Python Charting Tutorial](https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/time-series-visualization-with-altair/index.ipynb). 

La première fonction est en fait utilisée pour créer une fonction dans un petit bout de python. Le code ci-dessous définit une fonction réutilisable qui peut effectuer la tâche de réduction pour différents ensembles de données. La fonction accepte des arguments tels que l'échelle et la méthode de réduction pour paramétrer l'opération pour chaque analyse particulière.

Ne vous inquiétez pas trop d'essayer de comprendre *comment* les fonctions fonctionnent. Pour nos besoins, nous avons juste besoin de savoir qu'elles fonctionnent (elles le font) et que nous pouvons les réutiliser pour convertir des collections d'images en tableaux.


In [ ]:
# Define a function (and embedded function) for reducing a region for charting
# See the function descriptions defined in red within the functions

def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.median(),
                                  scale=500,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  """

  def reduce_region_function(img):
    """
      Function returns
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function


Maintenant que nous avons défini nos fonctions, nous pouvons les utiliser sur une collection d'images et une région afin d'obtenir un tableau que nous pouvons utiliser comme entrée pour certaines fonctions de la bibliothèque graphique.

In [ ]:
# Load MODIS Ocean Colour Image Collection
modisOceanColor = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI');

# Create a collection from the SST (sea surface temperature) band
sst = modisOceanColor.select(['sst'])

# Define an area of interest (aoi)
# We will look at Lake Erie 
aoi = ee.Geometry.Polygon([[-83.594,41.326],
                           [-78.782,41.326],
                           [-78.782,42.972],
                           [-83.594,42.972],
                           [-83.594,41.326]])


In [ ]:
# Check out the AOI if you want
# If you know where Lake Erie is, feel free to skip this cell
Map = geemap.Map()
Map.addLayer(aoi,{},'AOI')
Map

Le prochain morceau de code fait trois choses :
1. Assigner la fonction de réduction avec des paramètres spécifiques à la fonction, `reduce_sst`. Cela rendra plus simple le mappage de la fonction à travers la collection d'images.
2. Mapper la fonction sur la collection d'images `sst` pour réduire chaque image.
3. Filtre toutes les caractéristiques résultantes qui ont des valeurs calculées nulles (se produit lorsque tous les pixels dans un AOI sont masqués).

In [ ]:
# Use the above defined function to create a reducing function specific to the SST dataset
## We are going to leave most of the arguments as their defaults (see the above code)
## except geometry which we set to our aoi, the reducer which we will set as the median
reduce_sst = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.median())

# Convert the image collection to a feature collection with the reducer function
## this step just applies the function we created above (reduce_sst), with its defined inputs to our
## image collection
sst_stat_fc = ee.FeatureCollection(sst.map(reduce_sst))

# Filter out null values based on bandnames of the first image in the feature collection
sst_stat_fc = sst_stat_fc.filter(
    ee.Filter.notNull(sst.first().bandNames()))

L'étape suivante pour formater les données pour les graphiques est de créer une fonction `fc_to_dict` pour convertir la `ee.FeatureCollection` en `ee.Dictionary`. Ensuite, nous convertissons le `ee.Dictionary` en un dataframe pandas, qui est un format de tableau commun en python.

Rappelez-vous que Google Earth Engine fonctionne en envoyant du code entre votre ordinateur dans l'API python et les serveurs de Google pour le calcul. Afin de tracer des graphiques sans le fonctionnement du Chart.UI intégré de Google, nous devons transférer les données du serveur de Google (souvent appelé "côté serveur" dans la documentation) vers nos propres ordinateurs (appelés "côté client"), ce que nous faisons avec l'appel `.getInfo()`. 

Ce flux de travail est très utile pour intégrer les données de Google Earth Engine à d'autres bibliothèques python et fonctions de traitement de données, et pas seulement pour la création de graphiques. 

Mais revenons maintenant aux graphiques...

In [ ]:
# Define a function to transfer feature properties to a dictionary.
# We use this to convert our earth engine object (reduced image collection) to a 
# collection of properties and their values 

def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

Nous devons maintenant transférer les données, sous la forme d'un dictionnaire, sur notre propre ordinateur :

In [ ]:
# Convert feature collection to dictionary and use .getInfo() to transfer to our own computers
sst_dict = fc_to_dict(sst_stat_fc).getInfo()

Voyons maintenant à quoi ressemble le dictionnaire.

In [ ]:
# print normally
print(sst_dict,"\n") # \n tells it to print a new line

# print some of the dictionary with formatting
for prop in sst_dict.keys():
    print(prop + ':', sst_dict[prop][0:3] + ['...'])

Dans le format dictionnaire, les listes de données sont stockées dans des propriétés au format `{property1: [liste de valeurs1], propriété2: [liste de valeurs2], ...}`. Ce format peut facilement être converti en un cadre de données en créant des colonnes à partir des propriétés. Pour ce faire, nous allons utiliser la fonction `.DataFrame()` de la bibliothèque pandas (que nous avons importée sous le nom de pd).


In [ ]:
# Convert the dictionary to a dataframe to make the information easier to interpret
sst_df = pd.DataFrame(sst_dict)

Voyons maintenant à quoi ressemble le cadre de données.

In [ ]:
sst_df

Les données sont plus faciles à lire et à manipuler lorsqu'elles sont dans le format dataframe (tableau). 

L'étape suivante consiste à nettoyer les données avant de les tracer. La propriété `millis` contient des informations sur la date et l'heure dans un format étrange. Nous allons donc la convertir dans un format plus intuitif, puis créer des variables année, mois, jour et jour de l'année (DOY).

In [ ]:
# Function to add date variables to DataFrame.
# This function just uses pandas functions on the millis column to create date time columns

def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df  

In [ ]:
# apply the date info function to the dataframe
sst_df = add_date_info(sst_df)

# Rename the sst column SST and remove the unnecessary millis and system:index columns
sst_df = sst_df.rename(columns={
    'sst': 'SST'
}).drop(columns=['millis', 'system:index'])

Voyons maintenant à quoi ressemble notre cadre de données nettoyé.

In [ ]:
sst_df

#### Tracer le cadre de données

La partie la plus difficile est terminée. Nous disposons maintenant de nos données réduites (c'est-à-dire résumées) sur les moteurs terrestres sous forme de tableau. Si vous le souhaitez, vous pouvez exporter le tableau au format .csv et créer un graphique simple dans Excel. Cependant, une fois que l'on s'y est habitué, la création de graphiques en python est excellente et dispose de tonnes de bibliothèques et de fonctions pour créer de magnifiques graphiques personnalisés. Comme nous travaillons dans un carnet de notes, nous pouvons même les rendre interactifs !

Nous n'avons pas filtré notre collection d'images par date, donc notre dataframe `sst_df` contient toutes les données incluses dans le MODIS Ocean Color Dataset, c'est-à-dire les valeurs médianes de SST pour notre Area Of Interest (AOI) depuis le 4 juillet 2002. Ce produit de données n'est pas quotidien, mais il contient quand même beaucoup de données que nous pouvons visualiser.

Commençons par créer un nuage de points dans matplotlib, que vous avez peut-être déjà utilisé. 

#### Matplotlib

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt

# Set up the subplots. This is your base.
fig, ax = plt.subplots(figsize=(14, 6))

# Add scatter plot with blue points that are 20% opaque. Use DOY on the x, and SST on the y axis
ax.scatter(sst_df['DOY'], sst_df['SST'],
           c='black', alpha=0.2)

# Add some parameters.
ax.set_title('Lake Erie Surface Temperature Cycle', fontsize=16) # set the title
ax.set_xlabel('Date', fontsize=14) # set the x-axis label
ax.set_ylabel('Temperature [C]', fontsize=14) # set the y-axis label
ax.set_ylim(-4, 30) # set the y-axis range
ax.grid(lw=0.2) #set the opacity of the background grid to 20%


plt.show()

Notre graphique nous montre à peu près ce à quoi nous nous attendons pour un lac de l'hémisphère nord ! Il semble y avoir quelques valeurs aberrantes, mais dans l'ensemble, la température de surface du lac suit le même schéma pour tous nos points de données.

Matplotlib est un standard qui est très personnalisable (et possède des tonnes de pages d'assistance sur Internet). Cependant, il existe d'autres bibliothèques qui permettent de créer plus facilement de belles visualisations.

Pour cet exercice, nous utiliserons Altair, mais si vous êtes intéressé, essayez également de découvrir seaborn ! 

#### Altair

Le graphique matplotlib que nous avons créé est joli, mais il ne présente qu'une partie des informations dont nous disposons car il ne sépare pas les différentes années. Essayons un autre graphique avec Altair, dans lequel les différentes années sont représentées par des couleurs différentes.

In [ ]:
# Set the chart base up
base = alt.Chart(sst_df).encode(
    # set x-axis with DOY (see explanation of Q in comments below)
    # the scale specifies the range of our x-axis
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370])),
    # set y-axis as SST and set range from -5 to 30
    y=alt.Y('SST:Q', scale=alt.Scale(domain=[-5, 30])),
    # Change the point colours by Year value using the viridis colour scheme
    color=alt.Color('Year:O', scale=alt.Scale(scheme='viridis')))

# add the points to the map as circles
points = base.mark_circle()

# set overall properties
(points).properties(width=600, height=350)

## Note: Altair uses Q and O to specify the kind of data
# Q = quantitative, or continuous data (like SST)
# O = ordinal, or categorical data (like Year)

Ce graphique nous aide maintenant à voir quelles sont les années où se trouvent les valeurs aberrantes. Il semble que les valeurs aberrantes proviennent de nombreuses années différentes - il n'y a pas de tendance évidente à ce qu'une année ait une TSM supérieure ou inférieure à la normale.

Comme nous l'avons mentionné, Altair permet également de créer des graphiques interactifs. Nous pouvons maintenant adapter le code ci-dessus pour le rendre interactif. Nous allons également ajouter des lignes qui relient nos points pour chaque année à notre graphique.

In [ ]:
# Set the highlight parameters
# When we hover our mouse over a point, we want all the points that share that year
# to be highlighted
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

# Set the base, this is the same as above.
base = alt.Chart(sst_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370])),
    y=alt.Y('SST:Q', scale=alt.Scale(domain=[-5, 30])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='turbo')))

# Add points - change the size based on the highlight condition we set above
points = base.mark_circle().encode(
    size=alt.condition(~highlight, alt.value(10), alt.value(15)),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('SST:Q', title='SST')
    ]).add_selection(highlight)

# Add lines - change the size based on the highlight condition
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(4)))

# Set the chart properties and add .interactive()
(points + lines).properties(width=600, height=350).interactive()

Le graphique ci-dessus est interactif - lorsque vous survolez une valeur, il vous donne des informations sur ce point. Vous pouvez également zoomer sur le graphique et le faire glisser vers le haut, le bas, la droite et la gauche.

Les cartes et graphiques interactifs sont un excellent moyen d'utiliser la fonctionnalité des ordinateurs portables. 

#### Tracer la tendance de la concentration en Chl-a pour 2015

Pour notre dernier exemple, nous allons modifier notre code ci-dessus pour examiner la tendance de la concentration en chlorophylle pour 2015. Il y a eu une grande prolifération d'algues en 2015 dans le lac Érié, ce que nous devrions être en mesure de voir si nous traçons les valeurs. Le code ci-dessous suit la même structure que celle que nous avons utilisée plus haut pour la SST, mais sans tous les commentaires afin que vous puissiez le voir dans un format plus condensé.

In [ ]:
##### SET UP IMAGE COLLECTION AND AOI #####

# Create a collection from the chlor_a (Chlorophyll-a concentration) band
chla = modisOceanColor.select(['chlor_a'])

# Define an area of interest (aoi)
aoi = ee.Geometry.Polygon([[-83.594,41.326],
                           [-78.782,41.326],
                           [-78.782,42.972],
                           [-83.594,42.972],
                           [-83.594,41.326]])


In [ ]:
##### REDUCE IMAGE COLL #####

# Create reducer function for mapping across image collection
reduce_chla = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.median())

# Map reducing function across image collection and convert it to feature collection
chla_stat_fc = ee.FeatureCollection(chla.map(reduce_chla))

# Clean up by removing null values
chla_stat_fc = chla_stat_fc.filter(
    ee.Filter.notNull(chla.first().bandNames()))

In [ ]:
##### CONVERT COLLECTION TO TABLE #####

# Convert ee feature collection to 'client-side' dictionary
chla_dict = fc_to_dict(chla_stat_fc).getInfo()

# Convert dictionary to dataframe and add date info
chla_df = pd.DataFrame(chla_dict)
chla_df = add_date_info(chla_df)

# Clean up data columns and remove unnecessary columns
chla_df = chla_df.drop(columns=['millis', 'system:index'])

In [ ]:
chla_df

Jusqu'à présent, la plupart de ce que nous avons fait est exactement la même chose que ce que nous avons fait pour la température de surface de la mer ci-dessus. Nous avons juste changé notre bande pour être `chlor_a` au lieu de `sst` et nous avons changé nos noms de variables pour refléter ce changement. 

Notre cadre de données contient toutes les données depuis 2002, mais nous ne voulons que les données de 2015. Nous ***aurions pu*** filtrer notre collection d'images au début, comme nous l'avons fait dans les laboratoires précédents, ce qui aurait nécessité moins de données car nous n'aurions pas eu besoin de convertir les valeurs de toutes les autres années en un dictionnaire et un cadre de données. Cependant, nous pouvons également filtrer notre cadre de données, ce qui nous donne un peu plus de flexibilité au cas où nous voudrions représenter d'autres années. Cela nous donne également la possibilité d'utiliser certaines fonctions pandas pour filtrer le tableau/le cadre de données.

In [ ]:
# Filter dataframe using loc
# This line creates a new dataframe that only include rows from the
# original df that have values of 2015 in the Year column

chla2015_df = chla_df.loc[chla_df['Year'] == 2015]

In [ ]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['chlor_a'], nearest=True)

base = alt.Chart(chla2015_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370]),title = "Day of the Year"),
    y=alt.Y('chlor_a:Q', scale=alt.Scale(domain=[0,15]),title='Chl-a Concentration'),
    # use the goldgreen color scheme and set the variable as Q (continuous)
    color=alt.Color('chlor_a:Q', scale=alt.Scale(scheme='goldgreen'))) 

points = base.mark_circle().encode(
    size=alt.condition(~highlight, alt.value(100), alt.value(120)),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('chlor_a:Q', title='Chl-a')
    ]).add_selection(highlight)

(points).properties(width=600, height=350).interactive()

Dans le graphique ci-dessus, nous n'avons pas eu besoin de colorer les points en fonction de la concentration en chl-a car cela n'apporte aucune information supplémentaire (nous voyons déjà le modèle de chl-a en fonction de leur position sur le graphique). Cependant, cela peut être un outil de visualisation utile pour mettre l'accent sur le changement des valeurs. Vous pouvez utiliser une seule couleur en modifiant le code ci-dessus pour la base en ceci :

```
base = alt.Chart(sst2015_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370]),title = "Jour de l'année"),
    y=alt.Y('chlor_a:Q', scale=alt.Scale(domain=[0,15]),title="Concentration de Chl-a"),
    color=alt.value('green')) 
```

Le graphique ci-dessus montre qu'il y a des valeurs de Chl-a assez élevées à la fin de l'été. Voyons maintenant à quoi ressemblent la réflectance de la surface et les valeurs de Chl-a sur une carte !

In [ ]:
SR_Aug2015 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2015-08-01', '2015-09-30')\
        .filterBounds(aoi)\
        .map(maskEmptyPixels)\
        .map(maskClouds)\
        .median()

SR_May2015 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2015-04-01', '2015-05-31')\
        .filterBounds(aoi)\
        .map(maskEmptyPixels)\
        .map(maskClouds)\
        .median()


Chla_Aug2015 = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select(['chlor_a'])\
        .filterDate('2015-08-01', '2015-09-30')\
        .filterBounds(aoi)\
        .median()

Chla_May2015 = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select(['chlor_a'])\
        .filterDate('2015-04-01', '2015-05-31')\
        .filterBounds(aoi)\
        .median()

SRvis = {'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
     'gain': 0.07,
     'gamma': 1.7
    }

Chlavis = {'min':0,'max':16,'palette': 'blue,teal,green'};

Map = geemap.Map()
Map.addLayer(SR_May2015,SRvis,'May 2015 Median SR')
Map.addLayer(SR_Aug2015,SRvis,'Aug 2015 Median SR')
Map.addLayer(Chla_May2015,Chlavis,'May 2015 Median Chla')
Map.addLayer(Chla_Aug2015,Chlavis,'Aug 2015 Median Chla')
Map.addLayerControl()
Map.centerObject(aoi,8)
Map

Nous pouvons définitivement voir une différence entre les réflectances de surface pour avril/mai et août/septembre mais les valeurs de Chl-a sont très différentes ! 

Cela s'explique par le fait que le Chl-a est calculé en utilisant plus que les seules bandes RVB et qu'il présente donc plus de variations que ce que nous pouvons discerner à partir des images en couleurs. 

## Questions à soumettre

[Total des points = 10]

Répondez aux questions suivantes dans l'espace prévu à cet effet. N'hésitez pas à ajouter du code et des cellules de texte supplémentaires si nécessaire. Assurez-vous de montrer l'ensemble de votre code.

**Dans tous les cas**, si vous utilisez des morceaux de code provenant d'une autre source (ce qui n'est pas une mauvaise chose), veillez à indiquer où vous les avez trouvés.

### Q1

Affichez sur une carte la valeur médiane des TSM de 2018 pour l'ensemble du globe. Utilisez `'palette' : 'navy,blue,turquoise'` dans vos paramètres de visualisation. Assurez-vous que vos valeurs minimales et maximales ont un sens.

In [ ]:
## Montrez votre code ici

### Q2

Créez un diagramme de dispersion montrant la médiane de la TSM en 2018 pour une zone de votre choix n'importe où dans le monde (au-dessus de l'eau, bien sûr). Vous pouvez choisir soit un point, soit un rectangle pour définir votre zone. Vous pouvez obtenir les latitudes et longitudes en cliquant sur une carte dans ce cahier ou dans google maps.

In [ ]:
## Montrez votre code ici

### Q3

Maintenant que vous avez utilisé le moteur Google Earth, qu'en pensez-vous ? L'avez-vous trouvé utile et/ou intéressant ? Quelles sont ses forces et ses faiblesses ? Y a-t-il des choses que vous avez trouvées particulièrement difficiles ? Y a-t-il des choses que vous avez remarquées et qui sont particulièrement intéressantes ? Décrivez votre expérience en phrases complètes.

// Écrivez votre réponse ici